Mahalanobis Distance and Outliers blog post at http://kldavenport.com

## Calculating the Mahalanobis Distance and detecting outliers
### 1. Generate two random 1-D arrays:¶

In [1]:
import numpy as np
import random

random.seed(59)
# as column vectors
x = np.random.poisson(5,10)
y = np.random.poisson(5,10)

# examine the shape of the numpy arrays
print(x, y)
print(x.shape)
print(y.shape)

[6 3 6 3 9 3 3 6 6 7] [4 1 3 5 5 6 2 3 7 7]
(10,)
(10,)


### 2. Estimate a covariance matrix for (x,y):
An examination of what numpy's cov expects: A 1-D or 2-D array containing multiple variables and observations. Each row of m represents a variable, and each column a single observation of all those variables. In the event we are dealing with column vectors instead of row vectors, the rowvar parameter can be set to 1.

Here we calculate the covariance matrix:

In [4]:
covariance_xy = np.cov(x,y, rowvar=0)
inv_covariance_xy = np.linalg.inv(covariance_xy)

# Examine inverse covariance matrix and shape
print("covariance_xy",covariance_xy)
print("inv_covariance_xy: ",inv_covariance_xy)
print("inv_covariance_xy.shape: ",inv_covariance_xy.shape)

covariance_xy [[ 4.4         1.6       ]
 [ 1.6         4.23333333]]
inv_covariance_xy:  [[ 0.26348548 -0.09958506]
 [-0.09958506  0.27385892]]
inv_covariance_xy.shape:  (2, 2)


### 3. Center each value by the mean:
Center each value by the mean by subtracting the mean from i in array x and y.

In [6]:
xy_mean = np.mean(x),np.mean(y)
x_diff = np.array([x_i - xy_mean[0] for x_i in x])
y_diff = np.array([y_i - xy_mean[1] for y_i in y])
print(x_diff.shape)
print(xy_mean)
print(x_diff)
print(y_diff)

(10,)
(5.0999999999999996, 4.5999999999999996)
[-1.1  0.9 -2.1  0.9 -0.1  0.9 -2.1  1.9  3.9 -3.1]
[-1.6 -0.6 -2.6  2.4  0.4  1.4 -1.6 -0.6 -1.6  4.4]


Join the x_diff and y_diff arrays into (10 x 2) array to be used in our formula later. Consider these coordinate pairs.

In [8]:
diff_xy = np.transpose([x_diff, y_diff])
print(diff_xy.shape)
print(diff_xy)

(10, 2)
[[-1.1 -1.6]
 [ 0.9 -0.6]
 [-2.1 -2.6]
 [ 0.9  2.4]
 [-0.1  0.4]
 [ 0.9  1.4]
 [-2.1 -1.6]
 [ 1.9 -0.6]
 [ 3.9 -1.6]
 [-3.1  4.4]]


#### 4. Formula building
As stated above: $ MD = \sqrt{  (x - \mu)^T {\rm \bf  \Sigma}^{-1} (x - \mu)}$, 
So to put together this formula we'll need the square root of the transpose of our _diffxy array, the inverse of our _covariancexy matrix which we've assigned to _inv_covariancexy, and our original _diffxy array. Examining their shapes below for one final check before performing dot multiplication (remember u•v = v•u and u•(v + w) = u•w + v•w):

In [9]:
print(np.transpose(diff_xy[1]).shape)
print(inv_covariance_xy.shape)
print(diff_xy.shape)

(2,)
(2, 2)
(10, 2)


In [10]:
md = []
for i in range(len(diff_xy)):
    md.append(np.sqrt(np.dot(np.dot(np.transpose(diff_xy[i]),inv_covariance_xy),diff_xy[i])))
md

[0.99054224754225884,
 0.46836195462913394,
 1.7128195744953745,
 1.2749163095190754,
 0.18478256292712919,
 0.84709702603089077,
 1.3565240433972998,
 0.8987583895608503,
 1.8748800963809118,
 2.3077008424488397]

### We can wrap all this sequential code up into one function as follows:¶

In [11]:
def MahalanobisDist(x, y):
    covariance_xy = np.cov(x,y, rowvar=0)
    inv_covariance_xy = np.linalg.inv(covariance_xy)
    xy_mean = np.mean(x),np.mean(y)
    x_diff = np.array([x_i - xy_mean[0] for x_i in x])
    y_diff = np.array([y_i - xy_mean[1] for y_i in y])
    diff_xy = np.transpose([x_diff, y_diff])
    
    md = []
    for i in range(len(diff_xy)):
        md.append(np.sqrt(np.dot(np.dot(np.transpose(diff_xy[i]),inv_covariance_xy),diff_xy[i])))
    return md

In [12]:
MahalanobisDist(x,y)

[0.99054224754225884,
 0.46836195462913394,
 1.7128195744953745,
 1.2749163095190754,
 0.18478256292712919,
 0.84709702603089077,
 1.3565240433972998,
 0.8987583895608503,
 1.8748800963809118,
 2.3077008424488397]

In [13]:
def MD_removeOutliers(x, y):
    MD = MahalanobisDist(x, y)
    threshold = np.mean(MD) * 1.5 # adjust 1.5 accordingly 
    nx, ny, outliers = [], [], []
    for i in range(len(MD)):
        if MD[i] <= threshold:
            nx.append(x[i])
            ny.append(y[i])
        else:
            outliers.append(i) # position of removed pair
    return (np.array(nx), np.array(ny), np.array(outliers))

In [16]:
print('x:', x)
print('y:', y)
MD_removeOutliers(x,y)

x: [4 6 3 6 5 6 3 7 9 2]
y: [3 4 2 7 5 6 3 4 3 9]


(array([4, 6, 3, 6, 5, 6, 3, 7]),
 array([3, 4, 2, 7, 5, 6, 3, 4]),
 array([8, 9]))

The pair in position 6 is removed which is (9,6) where MD = 2.230 Graphing them below:

### Plotting in R's ggplot2
Here we create a dataframe that both the R and Python versions of ggplot will utilize:

In [17]:
import pandas as pd

DF_diff_xy = pd.DataFrame(diff_xy)
DF_diff_xy.rename(columns = lambda x: str(x), inplace=True)
DF_diff_xy.rename(columns={"0": "X"}, inplace=True) # rename a dfcolumn   
DF_diff_xy.rename(columns={"1": "Y"}, inplace=True) # rename a dfcolumn 
DF_diff_xy

,X,Y
0,-1.1,-1.6
1,0.9,-0.6
2,-2.1,-2.6
3,0.9,2.4
4,-0.1,0.4
5,0.9,1.4
6,-2.1,-1.6
7,1.9,-0.6
8,3.9,-1.6
9,-3.1,4.4


In [18]:
#%load_ext rmagic
%reload_ext rmagic

C:\Users\toshiba\Anaconda3\lib\site-packages\IPython\extensions\rmagic.py:11: UserWarning: The rmagic extension in IPython has moved to `rpy2.ipython`, please see `rpy2` documentation.
  warnings.warn("The rmagic extension in IPython has moved to "


In [19]:
%%R -i DF_diff_xy # list object to be transferred to python here
install.packages("ggplot2") # Had to add this for some reason, shouldn't be necessary
library(ggplot2)
df = data.frame(DF_diff_xy)
plot = ggplot(df, aes(x = X, y = Y)) + 
geom_point(alpha = .8, color = 'dodgerblue',size = 5) +
geom_point(data=subset(df, Y >= 0.7 & X >= 4.1), color = 'red',size = 6) +
theme(axis.text.x = element_text(size= rel(1.5),angle=90, hjust=1)) +
ggtitle('Distance Pairs with outliers highlighted in red')
print(plot)

ERROR:root:Cell magic `%%R` not found.


### Plotting in the python ggplot port from yhat
2013/12/13 Update I can now specify a dataframe per geom, thanks JanSchulz :)
https://github.com/yhat/ggplot/issues/116#issuecomment-29842170



In [20]:
ggplot(DF_diff_xy, aes(x = 'X', y ='Y')) + \
    geom_point(alpha=1, size=100, color='dodgerblue') + \
    geom_point(data = DF_diff_xy[(DF_diff_xy.Y >= 0.7) & (DF_diff_xy.X >= 4.1)],alpha=1, size = 100, color='red')

NameError: name 'ggplot' is not defined